<a href="https://colab.research.google.com/github/benbaz-2/comp551/blob/main/A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_parquet("hf://datasets/google-research-datasets/go_emotions/raw/train-00000-of-00001.parquet")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
labels = df.columns[9:]

In [4]:
df1 = df[df[labels].sum(axis=1) == 1]

In [5]:
comments = df1['text'].tolist()
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(comments)

In [6]:
n = X.shape[0]
y = np.array(df1[labels])

py = np.sum(y, axis=0) / n
px = []

for f in range(y.shape[1]):
    y_f = y[:, f].T
    X_f = y_f @ X
    px.append(X_f / np.sum(y_f))
px = np.array(px)

In [8]:
px.shape, py.shape, X.shape, y.shape

((28, 30449), (28,), (171820, 30449), (171820, 28))